In [319]:
#preprocess
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

filename = 'dataset/{}_2018.csv'
df = [pd.read_csv(filename.format(i), sep = 'delimiter') for i in range(1,13)]


Justified Function

In [320]:
def justify(a, invalid_val=0, axis=1, side='left'):    
    """
    Justifies a 2D array

    Parameters
    ----------
    A : ndarray
        Input array to be justified
    axis : int
        Axis along which justification is to be made
    side : str
        Direction of justification. It could be 'left', 'right', 'up', 'down'
        It should be 'left' or 'right' for axis=1 and 'up' or 'down' for axis=0.

    """

    if invalid_val is np.nan:
        mask = pd.notnull(a)
    else:
        mask = a!=invalid_val
    justified_mask = np.sort(mask,axis=axis)
    if (side=='up') | (side=='left'):
        justified_mask = np.flip(justified_mask,axis=axis)
    out = np.full(a.shape, invalid_val, dtype=object) 
    if axis==1:
        out[justified_mask] = a[mask]
    else:
        out.T[justified_mask.T] = a.T[mask.T]
    return out

Setiap dataframe memiliki format yang berbeda, dan itu menyusahkan sekali jika di concat menjadi satu dataframe. Maka dari itu, proses cleaning tetap dilakukan terpisah.

### df[0] = Januari

In [321]:
df_januari = df[0]
df_januari = df_januari.apply(lambda x : x.astype(str).str.lower())

print(df_januari.head())

#Split column
df_januari = df_januari['TANGGAL,KOMODITI,HARGA'].str.split(',', expand = True)

#melihat baris dengan tanggal 02/01/2018
df_januari.loc[df_januari[0] == '02/01/2018']

                              TANGGAL,KOMODITI,HARGA
0                             02/01/2018,beras,11340
1                        02/01/2018,gula pasir,13740
2        02/01/2018,minyak goreng bimoli botol,14040
3               02/01/2018,minyak goreng curah,12388
4  02/01/2018,daging sapi, lokal, paha belakang,1...


,0,1,2,3,4,5
0,02/01/2018,beras,11340,None,None,None
1,02/01/2018,gula pasir,13740,None,None,None
2,02/01/2018,minyak goreng bimoli botol,14040,None,None,None
3,02/01/2018,minyak goreng curah,12388,None,None,None
4,02/01/2018,daging sapi,lokal,paha belakang,117660,None
5,02/01/2018,daging sapi,lokal,daging,has luar (sirloin),118333
6,02/01/2018,daging sapi,lokal,daging,sandung lamur (brisket),89167
7,02/01/2018,daging sapi,lokal,daging,tetelan,70833
8,02/01/2018,daging sapi,impor,beku,120000,None
9,02/01/2018,daging sapi,lokal,has dalam,143333,None


Kita melihat bahwa baris Daging Sapi memiliki kolom lebih dari tiga. Kita akan cek baris keseluruhannya.

In [322]:
datecol = df_januari.loc[:, 0]

In [323]:
#Shift
right_shift = df_januari.loc[:, 1:5]
right_shift = justify(right_shift.values, invalid_val=np.nan, side = 'right' )
right_shift = pd.DataFrame(right_shift)
print(right_shift.head(), '\n')

#Combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1)

right_shift = right_shift.drop([1, 2, 3], axis = 1)
print(right_shift.head(), '\n')

#Concatenate 2 dataframe
df_jan = pd.concat([datecol, right_shift], axis = 1)
df_jan.columns = ['tanggal', 'komoditas', 'harga']
df_jan.komoditas = df_jan.komoditas.str.lstrip()

#Change to datetime for tanggal, and int for harga
df_jan.tanggal = df_jan.tanggal.apply(pd.to_datetime, dayfirst = True)
df_jan.harga = df_jan.harga.astype('int64')

#create beras premium dataframe
df_premium = df_jan[df_jan['komoditas'] == 'beras']
df_premium.komoditas = df_premium.komoditas.replace('beras', 'beras premium')
df_premium.harga = df_premium.harga + 2000

#change beras to beras medium
df_jan.komoditas = df_jan.komoditas.replace('beras', 'beras medium')

#concat
df_jan = pd.concat([df_jan, df_premium], axis = 'rows')

     0            1       2                           3       4
0  NaN          NaN     NaN                       beras   11340
1  NaN          NaN     NaN                  gula pasir   13740
2  NaN          NaN     NaN  minyak goreng bimoli botol   14040
3  NaN          NaN     NaN         minyak goreng curah   12388
4  NaN  daging sapi   lokal               paha belakang  117660 

                                 0       4
0                            beras   11340
1                       gula pasir   13740
2       minyak goreng bimoli botol   14040
3              minyak goreng curah   12388
4  daging sapi lokal paha belakang  117660 



In [324]:
def satuan(df_series):
    '''
    Add string measurement unit
    '''
    kg_list = ['medium', 'premium', 
           'pasir', 'belakang', 'sirloin', 'brisket',
           'tetelan', 'beku', 'dalam', 'ayam ras',
           'pipilan', 'halus', 'eks impor',
           'kedelai lokal', 'segitiga',
           'kriting', 'besar', 'rawit merah',
           'rawit hijau', 'bawang merah', 
           'impor kating', 'impor honan', 
           'teri', 'kembung', 'hijau', 'tanah', 'pohon']
    ekor1 = 'boiler'
    ekor2_list = ['ayam kampung']
    gram_list = ['bata']
    liter_List = ['bimoli', 'curah']
    buah_list = ['manis kaleng merk bendera', 'manis kaleng merk indomilk',
             'bubuk kaleng merk bendera', 'bubuk kaleng merk indomilk',
             'kari ayam']
    butir_list = ['telur ayam kampung']
    
    df_series = pd.np.where(df_series.str.contains('|'.join(kg_list)), df_series + '/kg', 
        pd.np.where(df_series.str.contains(ekor1), df_series + '/ekor',
        pd.np.where(df_series.str.contains('ayam kampung'), df_series.replace({'ayam kampung': 'ayam kampung/ekor', 'telur ayam kampung' : 'telur ayam kampung/butir'}),
        pd.np.where(df_series.str.contains('bata'), 'garam beryodium bata/250 gr', 
        pd.np.where(df_series.str.contains('|'.join(liter_List)), df_series + '/liter', df_series + '/buah')))))
    
    return df_series

In [325]:
df_jan.komoditas = df_jan.apply(lambda df_series : satuan(df_jan['komoditas']))


In [326]:
df_jan.shape

(836, 3)

### df[1] = Februari

In [327]:
def convert(df_series):
    x =  df_series.replace({'lokal daging has luar (sirloin)/kg' : 'daging sapi lokal daging has luar (sirloin)/kg',
                            'lokal daging sandung lamur (brisket)/kg' : 'daging sapi lokal daging sandung lamur (brisket)/kg',
                            'lokal daging tetelan/kg' : 'daging sapi lokal daging tetelan/kg'}, regex = True)\
                  .str.replace('cabe', 'cabai', regex = True).str.replace('mie instan ', '')
    return x

In [328]:
df_februari = df[1]
df_februari = df_februari.apply(lambda x : x.astype(str).str.lower())
df_februari.head()

,"TANGGAL,KOMODITI,SATUAN,HARGA"
0,"01/02/2018,beras medium,kg,10.863"
1,"02/02/2018,beras medium,kg,10.863"
2,"05/02/2018,beras medium,kg,10.890"
3,"06/02/2018,beras medium,kg,11.170"
4,"07/02/2018,beras medium,kg,11.040"


In [329]:
#Split column
df_februari = df_februari['TANGGAL,KOMODITI,SATUAN,HARGA'].str.split(',', expand = True)

print(df_februari.head(10))

            0             1   2       3     4     5     6
0  01/02/2018  beras medium  kg  10.863  None  None  None
1  02/02/2018  beras medium  kg  10.863  None  None  None
2  05/02/2018  beras medium  kg  10.890  None  None  None
3  06/02/2018  beras medium  kg  11.170  None  None  None
4  07/02/2018  beras medium  kg  11.040  None  None  None
5  08/02/2018  beras medium  kg  11.000  None  None  None
6  09/02/2018  beras medium  kg  11.380  None  None  None
7  12/02/2018  beras medium  kg  11.270  None  None  None
8  13/02/2018  beras medium  kg  11.180  None  None  None
9  14/02/2018  beras medium  kg  11.620  None  None  None


In [330]:
#put date column in :datecol
datecol = df_februari.loc[:, 0]
datecol

#right shift all column except datecol
right_shift = justify(df_februari.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)

#combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)

right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')

#concatenate and drop
df_feb = pd.concat([datecol, right_shift], axis = 'columns')
df_feb.columns = ['tanggal', 'komoditas', 'harga']


#change tanggal to datetime, harga to int64
df_feb.tanggal = df_feb.tanggal.apply(pd.to_datetime, dayfirst = True)
df_feb.harga = df_feb.harga.str.replace('.', '').astype('int64')

print(df_feb.head(10), '\n')

     tanggal        komoditas  harga
0 2018-02-01  beras medium/kg  10863
1 2018-02-02  beras medium/kg  10863
2 2018-02-05  beras medium/kg  10890
3 2018-02-06  beras medium/kg  11170
4 2018-02-07  beras medium/kg  11040
5 2018-02-08  beras medium/kg  11000
6 2018-02-09  beras medium/kg  11380
7 2018-02-12  beras medium/kg  11270
8 2018-02-13  beras medium/kg  11180
9 2018-02-14  beras medium/kg  11620 



In [331]:
print(df_feb.komoditas.unique(), '\n')
df_feb.komoditas = df_feb.apply(lambda x : convert(df_feb.komoditas))
print(df_feb.komoditas.unique())

['beras medium/kg' 'beras premium/kg' 'gula pasir/kg'
 'minyak goreng bimoli botol/liter' 'minyak goreng curah/liter'
 'daging sapi lokal paha belakang/kg' 'lokal daging has luar (sirloin)/kg'
 'lokal daging sandung lamur (brisket)/kg' 'lokal daging tetelan/kg'
 'daging sapi impor beku/kg' 'daging sapi lokal has dalam/kg'
 'ayam boiler/ekor' 'ayam kampung/ekor' 'telur ayam ras/kg'
 'telur ayam kampung/butir' 'susu kental manis kaleng merk bendera/buah'
 'susu kental manis kaleng merk indomilk/buah'
 'susu bubuk kaleng merk bendera/buah'
 'susu bubuk kaleng merk indomilk/buah' 'jagung pipilan kecil/kg'
 'garam beryodium bata/250 gr' 'garam beryodium halus/kg'
 'tepung terigu segitiga biru/kg' 'kacang kedelai eks impor/kg'
 'kacang kedelai lokal/kg' 'indomie rasa kari ayam/buah'
 'cabai merah kriting/kg' 'cabai merah besar/kg' 'cabai rawit merah/kg'
 'cabai rawit hijau/kg' 'bawang merah (medium)/kg'
 'bawang putih impor kating/kg' 'bawang putih impor honan/kg'
 'ikan teri asin/kg' 'ikan 

In [332]:
df_feb.tanggal.value_counts()

2018-02-20    38
2018-02-23    38
2018-02-08    38
2018-02-19    38
2018-02-02    38
2018-02-13    38
2018-02-07    38
2018-02-01    38
2018-02-12    38
2018-02-06    38
2018-02-09    38
2018-02-28    38
2018-02-22    38
2018-02-05    38
2018-02-27    38
2018-02-21    38
2018-02-15    38
2018-02-26    38
2018-02-14    38
Name: tanggal, dtype: int64

### df[2] = df_mar

In [333]:
df_mar = df[2]
df_mar = df_mar.apply(lambda x : x.astype(str).str.lower())
df_mar.head()
df_mar.isnull().sum()

TANGGAL,KOMODITI,SATUAN,HARGA    0
dtype: int64

In [334]:
#Split column
df_mar = df_mar['TANGGAL,KOMODITI,SATUAN,HARGA'].str.split(',', expand = True)
print(df_mar.head(10))
df_mar.isnull().sum()

            0             1   2      3     4     5     6
0  01/03/2018  beras medium  kg  11675  None  None  None
1  02/03/2018  beras medium  kg  11075  None  None  None
2  05/03/2018  beras medium  kg  10825  None  None  None
3  06/03/2018  beras medium  kg  10660  None  None  None
4  07/03/2018  beras medium  kg  10760  None  None  None
5  08/03/2018  beras medium  kg  11310  None  None  None
6  09/03/2018  beras medium  kg  10760  None  None  None
7  12/03/2018  beras medium  kg  11160  None  None  None
8  13/03/2018  beras medium  kg  10960  None  None  None
9  14/03/2018  beras medium  kg  11160  None  None  None


0      0
1      0
2      0
3      0
4    630
5    672
6    735
dtype: int64

In [335]:
#put date column in :datecol
datecol = df_mar.loc[:, 0]
 

#right shift all column except datecol
right_shift = justify(df_mar.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)

#combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)

right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')

#concatenate and drop
df_mar = pd.concat([datecol, right_shift], axis = 'columns')
df_mar.columns = ['tanggal', 'komoditas', 'harga']

#change tanggal to datetime, harga to int64, replace some strips
df_mar.tanggal = df_mar.tanggal.apply(pd.to_datetime, dayfirst = True)
df_mar.komoditas = df_mar.komoditas.str.replace('"', '')
df_mar.harga = df_mar.harga.str.replace('.', '').astype('int64')
print(df_mar.head())

     tanggal        komoditas  harga
0 2018-03-01  beras medium/kg  11675
1 2018-03-02  beras medium/kg  11075
2 2018-03-05  beras medium/kg  10825
3 2018-03-06  beras medium/kg  10660
4 2018-03-07  beras medium/kg  10760


In [336]:
df_mar.tanggal.value_counts()

2018-03-20    38
2018-03-23    38
2018-03-08    38
2018-03-19    38
2018-03-02    38
2018-03-13    38
2018-03-07    38
2018-03-01    38
2018-03-29    38
2018-03-12    38
2018-03-06    38
2018-03-09    38
2018-03-28    38
2018-03-22    38
2018-03-05    38
2018-03-16    38
2018-03-27    38
2018-03-21    38
2018-03-15    38
2018-03-26    38
2018-03-14    38
Name: tanggal, dtype: int64

### df[3] = df_april

In [337]:
df_april = df[3]
df_april = df_april.apply(lambda x : x.astype(str).str.lower())
df_april.head()

,"tanggal,komoditi,satuan,harga"
0,"2018-04-02,beras medium,kg,11240"
1,"2018-04-03,beras medium,kg,11680"
2,"2018-04-04,beras medium,kg,10940"
3,"2018-04-05,beras medium,kg,11480"
4,"2018-04-06,beras medium,kg,11780"


In [338]:
#Split column
df_april = df_april['tanggal,komoditi,satuan,harga'].str.split(',', expand = True)
print(df_april.head(10), '\n')

            0             1   2      3
0  2018-04-02  beras medium  kg  11240
1  2018-04-03  beras medium  kg  11680
2  2018-04-04  beras medium  kg  10940
3  2018-04-05  beras medium  kg  11480
4  2018-04-06  beras medium  kg  11780
5  2018-04-09  beras medium  kg  11580
6  2018-04-10  beras medium  kg  10880
7  2018-04-11  beras medium  kg  11480
8  2018-04-12  beras medium  kg  11680
9  2018-04-13  beras medium  kg  11290 



In [339]:
#combine
df_april[1] = df_april[df_april.columns[[1,2]]].apply(lambda x: '/'.join(x), axis = 1)

In [340]:
#drop
df_april = df_april.drop(2, axis = 'columns')
df_april.columns = ['tanggal', 'komoditas', 'harga']
df_april.tanggal = df_april.tanggal.str.replace('-', '/')
df_april.tanggal.unique()

array(['2018/04/02', '2018/04/03', '2018/04/04', '2018/04/05',
       '2018/04/06', '2018/04/09', '2018/04/10', '2018/04/11',
       '2018/04/12', '2018/04/13', '2018/04/16', '2018/04/17',
       '2018/04/18', '2018/04/19', '2018/04/20', '2018/04/23',
       '2018/04/24', '2018/04/25', '2018/04/26', '2018/04/27',
       '2018/04/30'], dtype=object)

In [341]:
df_april.tanggal = df_april.tanggal.str.replace('-', '/')
df_april.tanggal = df_april.tanggal.apply(pd.to_datetime, dayfirst = True)
df_april.harga = df_april.harga.astype('int64')

In [342]:
print(df_april.komoditas.unique())

['beras medium/kg' 'beras premium/kg' 'gula pasir/kg'
 'minyak goreng bimoli botol/liter' 'minyak goreng curah/liter'
 'daging sapi lokal paha belakang/kg'
 'daging sapi lokal daging has luar (sirloin)/kg'
 'daging sapi lokal daging sandung lamur (brisket)/kg'
 'daging sapi lokal daging tetelan/kg' 'daging sapi impor beku/kg'
 'daging sapi lokal has dalam/kg' 'ayam boiler/ekor' 'ayam kampung/ekor'
 'telur ayam ras/kg' 'telur ayam kampung/butir'
 'susu kental manis kaleng merk bendera/buah'
 'susu kental manis kaleng merk indomilk/buah'
 'susu bubuk kaleng merk bendera/buah'
 'susu bubuk kaleng merk indomilk/buah' 'jagung pipilan kecil/kg'
 'garam beryodium bata/250 gr' 'garam beryodium halus/kg'
 'tepung terigu segitiga biru/kg' 'kacang kedelai eks impor/kg'
 'kacang kedelai lokal/kg' 'indomie rasa kari ayam/buah'
 'cabai merah kriting/kg' 'cabai merah besar/kg' 'cabai rawit merah/kg'
 'cabai rawit hijau/kg' 'bawang merah (medium)/kg'
 'bawang putih impor kating/kg' 'bawang putih impor

In [343]:
df_april.tanggal.value_counts()

2018-04-06    38
2018-04-20    38
2018-04-11    38
2018-04-05    38
2018-04-16    38
2018-04-27    38
2018-04-10    38
2018-04-04    38
2018-04-26    38
2018-04-09    38
2018-04-03    38
2018-04-23    38
2018-04-25    38
2018-04-19    38
2018-04-02    38
2018-04-30    38
2018-04-13    38
2018-04-24    38
2018-04-18    38
2018-04-12    38
2018-04-17    38
Name: tanggal, dtype: int64

### df[4] = df_mei

In [344]:
df_mei = df[4]
df_mei = df_mei.apply(lambda x : x.astype(str).str.lower())
df_mei.head()

,"TANGGAL,KOMODITI,SATUAN,HARGA"
0,"02/05/2018,beras medium,kg,10990"
1,"03/05/2018,beras medium,kg,11250"
2,"04/05/2018,beras medium,kg,11300"
3,"07/05/2018,beras medium,kg,11040"
4,"08/05/2018,beras medium,kg,11040"


In [345]:
#Split column
df_mei = df_mei['TANGGAL,KOMODITI,SATUAN,HARGA'].str.split(',', expand = True)
print(df_mei.head(10), '\n')

            0             1   2      3     4     5     6
0  02/05/2018  beras medium  kg  10990  None  None  None
1  03/05/2018  beras medium  kg  11250  None  None  None
2  04/05/2018  beras medium  kg  11300  None  None  None
3  07/05/2018  beras medium  kg  11040  None  None  None
4  08/05/2018  beras medium  kg  11040  None  None  None
5  09/05/2018  beras medium  kg  10940  None  None  None
6  11/05/2018  beras medium  kg  10780  None  None  None
7  14/05/2018  beras medium  kg  10480  None  None  None
8  15/05/2018  beras medium  kg  10380  None  None  None
9  16/05/2018  beras medium  kg  10720  None  None  None 



In [346]:
#put date column in :datecol
datecol = df_mei.loc[:, 0]

#right shift all column except datecol
right_shift = justify(df_mei.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)

#print(right_shift[4].value_counts(), '\n')
#print(right_shift[3].value_counts(), '\n')

#combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)
right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')

#concatenate and drop
df_mei = pd.concat([datecol, right_shift], axis = 'columns')
df_mei.columns = ['tanggal', 'komoditas', 'harga']
print(df_mei.head(10), '\n')

#change tanggal to datetime, harga to int64, replace some strips
df_mei.tanggal = df_mei.tanggal.apply(pd.to_datetime, dayfirst = True)
df_mei.harga = df_mei.harga.str.replace('.', '').astype('int64')
print(df_mei.head())


      tanggal        komoditas  harga
0  02/05/2018  beras medium/kg  10990
1  03/05/2018  beras medium/kg  11250
2  04/05/2018  beras medium/kg  11300
3  07/05/2018  beras medium/kg  11040
4  08/05/2018  beras medium/kg  11040
5  09/05/2018  beras medium/kg  10940
6  11/05/2018  beras medium/kg  10780
7  14/05/2018  beras medium/kg  10480
8  15/05/2018  beras medium/kg  10380
9  16/05/2018  beras medium/kg  10720 

     tanggal        komoditas  harga
0 2018-05-02  beras medium/kg  10990
1 2018-05-03  beras medium/kg  11250
2 2018-05-04  beras medium/kg  11300
3 2018-05-07  beras medium/kg  11040
4 2018-05-08  beras medium/kg  11040


In [347]:
df_mei.tanggal.value_counts()

2018-05-04    38
2018-05-21    38
2018-05-09    38
2018-05-03    38
2018-05-31    38
2018-05-14    38
2018-05-25    38
2018-05-08    38
2018-05-02    38
2018-05-30    38
2018-05-24    38
2018-05-07    38
2018-05-18    38
2018-05-23    38
2018-05-17    38
2018-05-28    38
2018-05-11    38
2018-05-22    38
2018-05-16    38
2018-05-15    38
Name: tanggal, dtype: int64

### df[5] = df_juni

In [348]:
df_juni = df[5]
df_juni = df_juni.apply(lambda x : x.astype(str).str.lower())
df_juni.head()

,"TANGGAL,KOMODITI,SATUAN,HARGA"
0,"04/06/2018,beras medium,kg,10780"
1,"05/06/2018,beras medium,kg,10820"
2,"06/06/2018,beras medium,kg,10970"
3,"07/06/2018,beras medium,kg,10920"
4,"08/06/2018,beras medium,kg,11030"


In [349]:
#Split column
df_juni = df_juni['TANGGAL,KOMODITI,SATUAN,HARGA'].str.split(',', expand = True)
print(df_juni.head(10), '\n')

            0             1   2      3     4     5     6
0  04/06/2018  beras medium  kg  10780  None  None  None
1  05/06/2018  beras medium  kg  10820  None  None  None
2  06/06/2018  beras medium  kg  10970  None  None  None
3  07/06/2018  beras medium  kg  10920  None  None  None
4  08/06/2018  beras medium  kg  11030  None  None  None
5  21/06/2018  beras medium  kg  10920  None  None  None
6  22/06/2018  beras medium  kg  11080  None  None  None
7  25/06/2018  beras medium  kg  10720  None  None  None
8  26/06/2018  beras medium  kg  10820  None  None  None
9  28/06/2018  beras medium  kg  10920  None  None  None 



In [350]:
#put date column in :datecol
datecol = df_juni.loc[:, 0]

#right shift all column except datecol
right_shift = justify(df_juni.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)
#print(right_shift[4].value_counts(), '\n')
#print(right_shift[3].value_counts(), '\n')

#combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)
right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')

#concatenate and drop
df_juni = pd.concat([datecol, right_shift], axis = 'columns')
df_juni.columns = ['tanggal', 'komoditas', 'harga']
print(df_juni.head(10), '\n')

#change tanggal to datetime, harga to int64, replace some strips
df_juni.tanggal = df_juni.tanggal.apply(pd.to_datetime, dayfirst = True)
df_juni.harga = df_juni.harga.astype('int64')

print(df_juni.head())


      tanggal        komoditas  harga
0  04/06/2018  beras medium/kg  10780
1  05/06/2018  beras medium/kg  10820
2  06/06/2018  beras medium/kg  10970
3  07/06/2018  beras medium/kg  10920
4  08/06/2018  beras medium/kg  11030
5  21/06/2018  beras medium/kg  10920
6  22/06/2018  beras medium/kg  11080
7  25/06/2018  beras medium/kg  10720
8  26/06/2018  beras medium/kg  10820
9  28/06/2018  beras medium/kg  10920 

     tanggal        komoditas  harga
0 2018-06-04  beras medium/kg  10780
1 2018-06-05  beras medium/kg  10820
2 2018-06-06  beras medium/kg  10970
3 2018-06-07  beras medium/kg  10920
4 2018-06-08  beras medium/kg  11030


In [351]:
df_juni.tanggal.value_counts()

2018-06-04    38
2018-06-21    38
2018-06-07    38
2018-06-05    38
2018-06-22    38
2018-06-08    38
2018-06-25    38
2018-06-28    38
2018-06-06    38
2018-06-26    38
2018-06-29    38
Name: tanggal, dtype: int64

### df[6] = Juli

In [352]:
df_juli = df[6]
df_juli = df_juli.apply(lambda x : x.astype(str).str.lower())
df_juli.head()

,"tanggal,komoditi,satuan,harga"
0,"2018-07-02,beras medium,kg,10820"
1,"2018-07-03,beras medium,kg,10980"
2,"2018-07-04,beras medium,kg,10980"
3,"2018-07-05,beras medium,kg,10720"
4,"2018-07-06,beras medium,kg,10820"


In [353]:
#Split column
df_juli = df_juli['tanggal,komoditi,satuan,harga'].str.split(',', expand = True)
df_juli[2] = df_juli[2].str.lower()
print(df_juli.head(10), '\n')

            0             1   2      3
0  2018-07-02  beras medium  kg  10820
1  2018-07-03  beras medium  kg  10980
2  2018-07-04  beras medium  kg  10980
3  2018-07-05  beras medium  kg  10720
4  2018-07-06  beras medium  kg  10820
5  2018-07-09  beras medium  kg  10900
6  2018-07-10  beras medium  kg  10770
7  2018-07-11  beras medium  kg  10720
8  2018-07-12  beras medium  kg  10680
9  2018-07-13  beras medium  kg  10580 



In [354]:
df_juli[1].nunique()

38

In [355]:
#combine
df_juli[1] = df_juli[df_juli.columns[[1,2]]].apply(lambda x: '/'.join(x), axis = 1)

In [356]:
#drop
df_juli = df_juli.drop(2, axis = 'columns')
df_juli.columns = ['tanggal', 'komoditas', 'harga']
df_juli.tanggal = df_juli.tanggal.str.replace('-', '/')

#change tanggal to datetime, harga to int64, replace some strips
df_juli.tanggal = df_juli.tanggal.apply(pd.to_datetime, dayfirst = True)
df_juli.harga = df_juli.harga.astype('int64')


In [357]:
df_juli.tanggal.value_counts()

2018-07-05    38
2018-07-11    38
2018-07-27    38
2018-07-10    38
2018-07-04    38
2018-07-26    38
2018-07-09    38
2018-07-20    38
2018-07-03    38
2018-07-31    38
2018-07-25    38
2018-07-19    38
2018-07-02    38
2018-07-30    38
2018-07-13    38
2018-07-24    38
2018-07-18    38
2018-07-12    38
2018-07-23    38
2018-07-06    38
2018-07-17    38
2018-07-16    38
Name: tanggal, dtype: int64

### df[7] = df_agu

In [358]:
df_agu = df[7]
df_agu = df_agu.apply(lambda x : x.astype(str).str.lower())
df_agu.head()

,"tanggal,komoditi,satuan,harga"
0,"2018-08-01,beras medium,kg,10800"
1,"2018-08-02,beras medium,kg,10900"
2,"2018-08-03,beras medium,kg,10550"
3,"2018-08-06,beras medium,kg,10600"
4,"2018-08-07,beras medium,kg,10600"


In [359]:
#Split column
df_agu = df_agu['tanggal,komoditi,satuan,harga'].str.split(',', expand = True)
df_agu[2] = df_agu[2].str.lower()
print(df_agu.head(10), '\n')

            0             1   2      3     4     5     6
0  2018-08-01  beras medium  kg  10800  None  None  None
1  2018-08-02  beras medium  kg  10900  None  None  None
2  2018-08-03  beras medium  kg  10550  None  None  None
3  2018-08-06  beras medium  kg  10600  None  None  None
4  2018-08-07  beras medium  kg  10600  None  None  None
5  2018-08-08  beras medium  kg  10840  None  None  None
6  2018-08-09  beras medium  kg  10840  None  None  None
7  2018-08-10  beras medium  kg  10890  None  None  None
8  2018-08-13  beras medium  kg  10890  None  None  None
9  2018-08-14  beras medium  kg  10990  None  None  None 



In [360]:
#put date column in :datecol
datecol = df_agu.loc[:, 0]
datecol = datecol.str.replace('-', '/')

#right shift all column except datecol
right_shift = justify(df_agu.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)

#combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)
right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')

#concatenate and drop
df_agu = pd.concat([datecol, right_shift], axis = 'columns')
df_agu.columns = ['tanggal', 'komoditas', 'harga']
print(df_agu.head(10), '\n')

#change tanggal to datetime, harga to int64, replace some strips
df_agu.komoditas = df_agu.komoditas.str.replace('"', '')
df_agu.tanggal = df_agu.tanggal.apply(pd.to_datetime, dayfirst = True)
df_agu.harga = df_agu.harga.astype('int64')

print(df_agu.head())

      tanggal        komoditas  harga
0  2018/08/01  beras medium/kg  10800
1  2018/08/02  beras medium/kg  10900
2  2018/08/03  beras medium/kg  10550
3  2018/08/06  beras medium/kg  10600
4  2018/08/07  beras medium/kg  10600
5  2018/08/08  beras medium/kg  10840
6  2018/08/09  beras medium/kg  10840
7  2018/08/10  beras medium/kg  10890
8  2018/08/13  beras medium/kg  10890
9  2018/08/14  beras medium/kg  10990 

     tanggal        komoditas  harga
0 2018-08-01  beras medium/kg  10800
1 2018-08-02  beras medium/kg  10900
2 2018-08-03  beras medium/kg  10550
3 2018-08-06  beras medium/kg  10600
4 2018-08-07  beras medium/kg  10600


In [361]:
df_agu.tanggal.value_counts()

2018-08-08    38
2018-08-22    38
2018-08-30    38
2018-08-13    38
2018-08-07    38
2018-08-01    38
2018-08-29    38
2018-08-23    38
2018-08-06    38
2018-08-28    38
2018-08-16    38
2018-08-14    38
2018-08-27    38
2018-08-10    38
2018-08-21    38
2018-08-15    38
2018-08-09    38
2018-08-20    38
2018-08-03    38
2018-08-31    38
2018-08-02    38
Name: tanggal, dtype: int64

### df[8] = df_sep

In [362]:
df_sep = df[8]
df_sep = df_sep.apply(lambda x : x.astype(str).str.lower())
df_sep.head()

,"tanggal,komoditi,satuan,harga"
0,"2018-09-03,beras medium,kg,10840"
1,"2018-09-04,beras medium,kg,10840"
2,"2018-09-05,beras medium,kg,10540"
3,"2018-09-06,beras medium,kg,10540"
4,"2018-09-07,beras medium,kg,10640"


In [363]:
#Split column
df_sep = df_sep['tanggal,komoditi,satuan,harga'].str.split(',', expand = True)
print(df_sep.head(10), '\n')

            0             1   2      3     4     5     6
0  2018-09-03  beras medium  kg  10840  None  None  None
1  2018-09-04  beras medium  kg  10840  None  None  None
2  2018-09-05  beras medium  kg  10540  None  None  None
3  2018-09-06  beras medium  kg  10540  None  None  None
4  2018-09-07  beras medium  kg  10640  None  None  None
5  2018-09-10  beras medium  kg  10640  None  None  None
6  2018-09-12  beras medium  kg  10640  None  None  None
7  2018-09-13  beras medium  kg  10540  None  None  None
8  2018-09-14  beras medium  kg  10640  None  None  None
9  2018-09-17  beras medium  kg  10840  None  None  None 



In [364]:
#put date column in :datecol
datecol = df_sep.loc[:, 0]
datecol = datecol.str.replace('-', '/')


#right shift all column except datecol
right_shift = justify(df_sep.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)

#combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)
right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')

#concatenate and drop
df_sep = pd.concat([datecol, right_shift], axis = 'columns')
df_sep.columns = ['tanggal', 'komoditas', 'harga']

#change tanggal to datetime, harga to int64, replace some strips
df_sep.komoditas = df_sep.komoditas.str.replace('"', '')
df_sep.tanggal = df_sep.tanggal.apply(pd.to_datetime, dayfirst = True)
df_sep.harga = df_sep.harga.astype('int64')

print(df_sep.head())

#Pivoting
sep_pivot = df_sep.pivot(index = 'tanggal', columns = 'komoditas', values = 'harga')

     tanggal        komoditas  harga
0 2018-09-03  beras medium/kg  10840
1 2018-09-04  beras medium/kg  10840
2 2018-09-05  beras medium/kg  10540
3 2018-09-06  beras medium/kg  10540
4 2018-09-07  beras medium/kg  10640


In [365]:
sep_pivot.head()

komoditas,ayam boiler/ekor,ayam kampung/ekor,bawang merah (medium)/kg,bawang putih impor honan/kg,bawang putih impor kating/kg,beras medium/kg,beras premium/kg,cabai merah besar/kg,cabai merah kriting/kg,cabai rawit hijau/kg,...,lokal daging tetelan/kg,minyak goreng bimoli botol/liter,minyak goreng curah/liter,susu bubuk kaleng merk bendera/buah,susu bubuk kaleng merk indomilk/buah,susu kental manis kaleng merk bendera/buah,susu kental manis kaleng merk indomilk/buah,telur ayam kampung/butir,telur ayam ras/kg,tepung terigu segitiga biru/kg
tanggal,,,,,,,,,,,,,,,,,,,,,
2018-09-03,35440,80000,27660,27500,29940,10840,12575,37660,31500,37500,...,71667,13200,12400,44175,42550,11300,10020,2320,23740,10240
2018-09-04,35940,82500,28460,27500,30440,10840,12575,37660,32500,36500,...,65000,13150,12300,44175,42550,11300,10020,2320,23690,10240
2018-09-05,35940,82500,26460,26500,29540,10540,12450,38660,28300,34500,...,70000,13100,12200,44175,42550,11300,9920,2330,23740,10240
2018-09-06,34940,82500,28160,27500,32840,10540,12388,39160,31900,37000,...,70000,13000,12300,44175,42550,11500,9920,2320,23540,10040
2018-09-07,34940,85000,28660,26500,32340,10640,12575,39160,31000,33500,...,71667,13100,12100,44175,42550,11300,9920,2370,23140,10140


In [366]:
df_sep.tanggal.value_counts()

2018-09-05    38
2018-09-25    38
2018-09-10    38
2018-09-21    38
2018-09-04    38
2018-09-26    38
2018-09-20    38
2018-09-03    38
2018-09-14    38
2018-09-19    38
2018-09-28    38
2018-09-13    38
2018-09-24    38
2018-09-07    38
2018-09-18    38
2018-09-12    38
2018-09-06    38
2018-09-17    38
2018-09-27    38
Name: tanggal, dtype: int64

### df[9] = df_okt

In [367]:
df_okt = df[9]
df_okt = df_okt.apply(lambda x : x.astype(str).str.lower())
df_okt.head()

,"tanggal ,komoditi,satuan,harga"
0,"2018-10-01,beras medium,kg,10740"
1,"2018-10-02,beras medium,kg,10740"
2,"2018-10-03,beras medium,kg,10690"
3,"2018-10-04,beras medium,kg,10640"
4,"2018-10-05,beras medium,kg,10540"


In [368]:
#Split column
df_okt = df_okt['tanggal ,komoditi,satuan,harga'].str.split(',', expand = True)
print(df_okt.head(10), '\n')

            0             1   2      3     4     5     6
0  2018-10-01  beras medium  kg  10740  None  None  None
1  2018-10-02  beras medium  kg  10740  None  None  None
2  2018-10-03  beras medium  kg  10690  None  None  None
3  2018-10-04  beras medium  kg  10640  None  None  None
4  2018-10-05  beras medium  kg  10540  None  None  None
5  2018-10-08  beras medium  kg  10540  None  None  None
6  2018-10-09  beras medium  kg  10540  None  None  None
7  2018-10-10  beras medium  kg  10740  None  None  None
8  2018-10-11  beras medium  kg  10940  None  None  None
9  2018-10-12  beras medium  kg  10740  None  None  None 



In [369]:
#put date column in :datecol
datecol = df_okt.loc[:, 0]
datecol = datecol.str.replace('-', '/')


#right shift all column except datecol
right_shift = justify(df_okt.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)

#combine and drop
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)
right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')

#concatenate and drop
df_okt = pd.concat([datecol, right_shift], axis = 'columns')
df_okt.columns = ['tanggal', 'komoditas', 'harga']
print(df_okt.head(10), '\n')

#change tanggal to datetime, harga to int64, replace some strips
df_okt.komoditas = df_okt.komoditas.str.replace('"', '')
df_okt.tanggal = df_okt.tanggal.apply(pd.to_datetime, dayfirst = True)
df_okt.harga = df_okt.harga.astype('int64')

print(df_okt.head())


      tanggal        komoditas  harga
0  2018/10/01  beras medium/kg  10740
1  2018/10/02  beras medium/kg  10740
2  2018/10/03  beras medium/kg  10690
3  2018/10/04  beras medium/kg  10640
4  2018/10/05  beras medium/kg  10540
5  2018/10/08  beras medium/kg  10540
6  2018/10/09  beras medium/kg  10540
7  2018/10/10  beras medium/kg  10740
8  2018/10/11  beras medium/kg  10940
9  2018/10/12  beras medium/kg  10740 

     tanggal        komoditas  harga
0 2018-10-01  beras medium/kg  10740
1 2018-10-02  beras medium/kg  10740
2 2018-10-03  beras medium/kg  10690
3 2018-10-04  beras medium/kg  10640
4 2018-10-05  beras medium/kg  10540


In [370]:
df_okt.nunique()

tanggal       23
komoditas     38
harga        341
dtype: int64

In [371]:
df_okt.tanggal.value_counts()

2018-10-09    38
2018-10-29    38
2018-10-31    38
2018-10-25    38
2018-10-08    38
2018-10-19    38
2018-10-02    38
2018-10-30    38
2018-10-24    38
2018-10-18    38
2018-10-01    38
2018-10-12    38
2018-10-26    38
2018-10-23    38
2018-10-17    38
2018-10-11    38
2018-10-22    38
2018-10-05    38
2018-10-16    38
2018-10-10    38
2018-10-04    38
2018-10-15    38
2018-10-03    38
Name: tanggal, dtype: int64

### df[10] = df_nov

In [372]:
df_nov = df[10]
df_nov = df_nov.apply(lambda x : x.astype(str).str.lower())
df_nov.head()

,"tanggal,bulan,tahun,komoditi,satuan,harga"
0,"1,november,2018,beras medium,kg,""10,860"""
1,"2,november,2018,beras medium,kg,""10,860"""
2,"5,november,2018,beras medium,kg,""10,760"""
3,"6,november,2018,beras medium,kg,""10,710"""
4,"7,november,2018,beras medium,kg,""10,760"""


In [373]:
#strip and split
df_nov.columns = [0]
df_nov[0] = df_nov[0].str.replace(',november,','/11/')

df_nov = df_nov[0].str.split(',', expand = True)
print(df_nov.head(10), '\n')

            0             1   2    3     4     5     6     7
0   1/11/2018  beras medium  kg  "10  860"  None  None  None
1   2/11/2018  beras medium  kg  "10  860"  None  None  None
2   5/11/2018  beras medium  kg  "10  760"  None  None  None
3   6/11/2018  beras medium  kg  "10  710"  None  None  None
4   7/11/2018  beras medium  kg  "10  760"  None  None  None
5   8/11/2018  beras medium  kg  "10  560"  None  None  None
6   9/11/2018  beras medium  kg  "10  760"  None  None  None
7  12/11/2018  beras medium  kg  "10  860"  None  None  None
8  13/11/2018  beras medium  kg  "10  540"  None  None  None
9  14/11/2018  beras medium  kg  "10  490"  None  None  None 



In [374]:
#put date column in :datecol
datecol = df_nov.loc[:, 0]

#right shift all column except datecol, concat and drop last column
right_shift = justify(df_nov.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)
right_shift[5] = right_shift[5] + right_shift[6]
right_shift = right_shift.drop(6, axis = 'columns')


#combine, drop, concat
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)
right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')


df_nov = pd.concat([datecol, right_shift], axis = 'columns')
df_nov.columns = ['tanggal', 'komoditas', 'harga']

#change tanggal to datetime, harga to int64, replace some strips
df_nov.tanggal = df_nov.tanggal.apply(pd.to_datetime, dayfirst = True)
df_nov.harga = df_nov.harga.str.replace('"', '')
df_nov.komoditas = df_nov.komoditas.str.replace('"', '')

print(df_nov.head())


     tanggal        komoditas  harga
0 2018-11-01  beras medium/kg  10860
1 2018-11-02  beras medium/kg  10860
2 2018-11-05  beras medium/kg  10760
3 2018-11-06  beras medium/kg  10710
4 2018-11-07  beras medium/kg  10760


In [375]:
df_nov.nunique()

tanggal       21
komoditas     38
harga        354
dtype: int64

In [376]:
df_nov.tanggal.value_counts()

2018-11-06    38
2018-11-14    38
2018-11-22    38
2018-11-05    38
2018-11-16    38
2018-11-27    38
2018-11-21    38
2018-11-15    38
2018-11-26    38
2018-11-09    38
2018-11-08    38
2018-11-23    38
2018-11-19    38
2018-11-02    38
2018-11-30    38
2018-11-13    38
2018-11-07    38
2018-11-01    38
2018-11-29    38
2018-11-12    38
2018-11-28    38
Name: tanggal, dtype: int64

### df[11] = df_des

In [377]:
df_des = df[11]
df_des = df_des.apply(lambda x : x.astype(str).str.lower())
df_des.head()

,"TANGGAL,BULAN,TAHUN,KOMODITI,SATUAN,HARGA"
0,"3,desember,2018,beras medium,kg,10790"
1,"4,desember,2018,beras medium,kg,10590"
2,"5,desember,2018,beras medium,kg,10500"
3,"6,desember,2018,beras medium,kg,10450"
4,"7,desember,2018,beras medium,kg,10500"


In [378]:
df_des = df_des[~(df_des['TANGGAL,BULAN,TAHUN,KOMODITI,SATUAN,HARGA'] == ',,,,,')]

In [379]:
#strip and split
df_des.columns = [0]
df_des[0] = df_des[0].str.replace(',desember,', '/12/')
print(df_des.head(10), '\n')
df_des = df_des[0].str.split(',', expand = True)

                                  0
0   3/12/2018,beras medium,kg,10790
1   4/12/2018,beras medium,kg,10590
2   5/12/2018,beras medium,kg,10500
3   6/12/2018,beras medium,kg,10450
4   7/12/2018,beras medium,kg,10500
5  10/12/2018,beras medium,kg,10500
6  11/12/2018,beras medium,kg,10350
7  12/12/2018,beras medium,kg,10590
8  13/12/2018,beras medium,kg,10540
9  14/12/2018,beras medium,kg,10590 



In [380]:
#put date column in :datecol
datecol = df_des.loc[:, 0]


In [381]:
#right shift all column except datecol, concat and drop last column
right_shift = justify(df_des.loc[:, 1:].values, invalid_val = np.nan, side = 'right')
right_shift = pd.DataFrame(right_shift)
right_shift

#combine, drop, concat
right_shift[0] = right_shift[right_shift.columns[1:4]]\
                            .apply(lambda x: ''.join(x.dropna().astype(str)),
                                  axis = 1).str.lstrip()
right_shift[0] = right_shift[right_shift.columns[[0,4]]].apply(lambda x: '/'.join(x), axis = 1)
right_shift = right_shift.drop([1, 2, 3, 4], axis = 'columns')
right_shift.head()
df_des = pd.concat([datecol, right_shift], axis = 'columns')
df_des.columns = ['tanggal', 'komoditas', 'harga']
df_des.harga = pd.to_numeric(df_des.harga).round(2)

#change tanggal to datetime, harga to int64, replace some strips
df_des.tanggal = df_des.tanggal.apply(pd.to_datetime, dayfirst = True)
df_des.komoditas = df_des.komoditas.str.replace('"', '')

print(df_des.head())

     tanggal        komoditas    harga
0 2018-12-03  beras medium/kg  10790.0
1 2018-12-04  beras medium/kg  10590.0
2 2018-12-05  beras medium/kg  10500.0
3 2018-12-06  beras medium/kg  10450.0
4 2018-12-07  beras medium/kg  10500.0


### Let's concat!





In [382]:
list_df = [df_jan, df_feb, df_mar, 
           df_april, df_mei, df_juni, 
           df_juli, df_agu, df_sep, 
           df_okt, df_nov, df_des]

df_pangan = pd.concat(list_df, axis = 'rows', ignore_index=True, )
print(df_pangan.tail(), '\n')
print(df_pangan.info())

#drop row with nan value
print(df_pangan.info())
df_pangan.tanggal.value_counts()

        tanggal        komoditas harga
9077 2018-12-21  ketela pohon/kg  5340
9078 2018-12-26  ketela pohon/kg  5340
9079 2018-12-27  ketela pohon/kg  5340
9080 2018-12-28  ketela pohon/kg  5340
9081 2018-12-31  ketela pohon/kg  5340 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9082 entries, 0 to 9081
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   tanggal    9082 non-null   datetime64[ns]
 1   komoditas  9082 non-null   object        
 2   harga      9082 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 213.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9082 entries, 0 to 9081
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   tanggal    9082 non-null   datetime64[ns]
 1   komoditas  9082 non-null   object        
 2   harga      9082 non-null   object        
dtypes: datetime

2018-05-21    38
2018-12-03    38
2018-02-08    38
2018-07-16    38
2018-12-21    38
              ..
2018-10-01    38
2018-01-15    38
2018-06-22    38
2018-11-27    38
2018-08-30    38
Name: tanggal, Length: 239, dtype: int64

In [383]:
df_pangan.komoditas.nunique()

49

In [384]:
#replacing
df_pangan.komoditas[df_pangan.komoditas.str.contains('sirloin')] = 'daging sapi lokal daging has luar (sirloin)/kg'
df_pangan.komoditas[df_pangan.komoditas.str.contains('brisket')] = 'daging sapi lokal daging sandung lamur (brisket)/kg'
df_pangan.komoditas[df_pangan.komoditas.str.contains('tetelan/kg')] = 'daging sapi lokal daging tetelan/kg'
df_pangan.komoditas = df_pangan.komoditas.str.replace('cabe', 'cabai')

d = {'keriting/buah' : 'keriting/kg', 'kriting/kg' : 'keriting/kg',
    'honan/kg' : '(honan)/kg', 'bata/251' : 'bata/250', 'kating/kg' : '(kating)/kg', 'mie instan ' : ''}
df_pangan.komoditas = df_pangan.komoditas.replace(d, regex = True)
df_pangan.komoditas = df_pangan.komoditas.str.replace('(honan)/buah', '(honan)/kg', regex = False)\
                               .str.replace('(kating)/buah','(kating)/kg', regex = False)

In [385]:
print(df_pangan.komoditas.nunique(), '\n')

38 



In [386]:
#reformat datetime
df_pangan.tanggal = pd.to_datetime(df_pangan.tanggal, infer_datetime_format= True, dayfirst=True)

df_pangan.tail()

,tanggal,komoditas,harga
9077,2018-12-21,ketela pohon/kg,5340
9078,2018-12-26,ketela pohon/kg,5340
9079,2018-12-27,ketela pohon/kg,5340
9080,2018-12-28,ketela pohon/kg,5340
9081,2018-12-31,ketela pohon/kg,5340


In [387]:
df_pangan = df_pangan.drop_duplicates()
df_pangan.harga = df_pangan.harga.astype(int)

In [388]:
df_pangan = df_pangan.drop_duplicates(ignore_index=True)
df_pangan[df_pangan.duplicated()]

,tanggal,komoditas,harga


In [389]:
df_pangan = df_pangan.set_index('tanggal')

In [390]:
pangan_pivot = df_pangan.pivot_table(index = 'tanggal', columns = 'komoditas', values = 'harga')
pangan_pivot

komoditas,ayam boiler/ekor,ayam kampung/ekor,bawang merah (medium)/kg,bawang putih impor (honan)/kg,bawang putih impor (kating)/kg,beras medium/kg,beras premium/kg,cabai merah besar/kg,cabai merah keriting/kg,cabai rawit hijau/kg,...,ketela pohon/kg,minyak goreng bimoli botol/liter,minyak goreng curah/liter,susu bubuk kaleng merk bendera/buah,susu bubuk kaleng merk indomilk/buah,susu kental manis kaleng merk bendera/buah,susu kental manis kaleng merk indomilk/buah,telur ayam kampung/butir,telur ayam ras/kg,tepung terigu segitiga biru/kg
tanggal,,,,,,,,,,,,,,,,,,,,,
2018-01-02,39740,55075,26800,25000,28560,11340,13340,37260,37140,34800,...,5060,14040,12388,43667,42000,12340,10300,2100,26150,10160
2018-01-03,38940,55075,27800,25000,30260,11440,13440,37260,38640,35300,...,5060,14240,12450,43667,42000,12240,10400,2150,26200,10360
2018-01-04,37440,53825,28400,27500,28660,11490,13490,39260,39340,41400,...,5060,14240,12575,43667,42000,11940,10600,2100,26300,9860
2018-01-05,36940,78825,27900,27500,28960,11740,13740,40260,42340,40400,...,4960,14040,12200,43667,42000,11940,10300,2120,26450,9610
2018-01-08,36940,77575,27900,27500,28960,11740,13740,40260,42340,40400,...,4960,13440,12200,43667,42000,11940,10300,2120,26450,9610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,38860,86250,30260,26500,28160,13187,12437,42200,33660,27240,...,5340,13340,11260,44375,42325,11510,9820,2340,26500,9740
2018-12-26,39060,83750,30960,26500,27860,13187,12375,43700,34060,27440,...,5340,13240,11560,44375,42325,11460,9820,2320,26600,9740
2018-12-27,40200,95000,30740,25500,27260,13250,12700,40560,32800,28440,...,5340,13560,11600,44375,42450,12100,9960,2340,26740,9700


In [391]:
#pangan_clean = pangan_clean.set_index('tanggal').reindex(method='bfill')

#to_csv
df_pangan.to_csv('pangan_clean.csv')


In [392]:
pangan_pivot.to_excel('pangan_pivot.xls')

In [393]:
pangan_pivot.isnull().sum()

komoditas
ayam boiler/ekor                                       0
ayam kampung/ekor                                      0
bawang merah (medium)/kg                               0
bawang putih impor (honan)/kg                          0
bawang putih impor (kating)/kg                         0
beras medium/kg                                        0
beras premium/kg                                       0
cabai merah besar/kg                                   0
cabai merah keriting/kg                                0
cabai rawit hijau/kg                                   0
cabai rawit merah/kg                                   0
daging sapi impor beku/kg                              0
daging sapi lokal daging has luar (sirloin)/kg         0
daging sapi lokal daging sandung lamur (brisket)/kg    0
daging sapi lokal daging tetelan/kg                    0
daging sapi lokal has dalam/kg                         0
daging sapi lokal paha belakang/kg                     0
garam beryodium bata/